# 🥉 Bronze Layer — Raw Ingestion

**Annie's Magic Numbers Medallion Architecture**

This notebook ingests raw CSV files into the Bronze layer in Delta format.

### 🔐 CELL 0 — ADLS Gen2 Authentication (Storage Account Key)

In [ ]:
# ============================================================
# CELL 0 — Azure Data Lake Gen2 Authentication
# ============================================================
# This cell configures authentication so Databricks can access
# the ADLS Gen2 account using a Storage Account Key.

spark.conf.set(
    "fs.azure.account.key.anniedatalake123.dfs.core.windows.net",
    "<PASTE_STORAGE_ACCOUNT_KEY_1_HERE>"
)

### 🟦 CELL 1 — Azure Data Lake Base Paths

In [ ]:
# ============================================================
# CELL 1 — Azure Data Lake Gen2 Base Paths
# ============================================================
container_name = "annie-data"
storage_account = "anniedatalake123"

base_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/"
raw_path = base_path + "raw/"
bronze_path = base_path + "bronze/"

### 🟦 CELL 2 — Validate RAW Zone Accessibility

In [ ]:
# ============================================================
# CELL 2 — Validate RAW Zone Accessibility
# ============================================================
dbutils.fs.ls(raw_path)

### 🟦 CELL 3 — Generic CSV Reader Function

In [ ]:
# ============================================================
# CELL 3 — Generic CSV Reader Function
# ============================================================
def read_csv(filename):
    return (
        spark.read
             .option("header", True)
             .option("inferSchema", True)
             .csv(raw_path + filename)
    )

### 🟦 CELL 4 — Load RAW CSV Files into DataFrames

In [ ]:
# ============================================================
# CELL 4 — Load RAW CSV Files into DataFrames
# ============================================================
sales_df = read_csv("SalesFINAL12312016.csv")
purchases_df = read_csv("PurchasesFINAL12312016.csv")
prices_df = read_csv("2017PurchasePricesDec.csv")
begin_inventory_df = read_csv("BegInvFINAL12312016.csv")
end_inventory_df = read_csv("EndInvFINAL12312016.csv")
invoices_df = read_csv("InvoicePurchases12312016.csv")

### 🟦 CELL 5 — Data Inspection & Schema Validation

In [ ]:
# ============================================================
# CELL 5 — Data Inspection & Schema Validation
# ============================================================
display(sales_df)
display(purchases_df)
display(prices_df)

### 🟦 CELL 6 — Bronze Delta Writer Function

In [ ]:
# ============================================================
# CELL 6 — Bronze Delta Writer Function
# ============================================================
def write_bronze(df, table_name):
    (
        df.write
          .format("delta")
          .mode("overwrite")
          .save(bronze_path + table_name)
    )

### 🟦 CELL 7 — Persist DataFrames to Bronze Layer

In [ ]:
# ============================================================
# CELL 7 — Persist DataFrames to the Bronze Layer
# ============================================================
write_bronze(sales_df, "sales")
write_bronze(purchases_df, "purchases")
write_bronze(prices_df, "prices")
write_bronze(begin_inventory_df, "begin_inventory")
write_bronze(end_inventory_df, "end_inventory")
write_bronze(invoices_df, "invoices")

### 🟦 CELL 8 — Validate Bronze Folder Structure

In [ ]:
# ============================================================
# CELL 8 — Validate Bronze Folder Structure
# ============================================================
dbutils.fs.ls(bronze_path)

### 🟦 CELL 9 — Validate Delta Read from Bronze

In [ ]:
# ============================================================
# CELL 9 — Validate Delta Read from Bronze
# ============================================================
sales_bronze_df = (
    spark.read
         .format("delta")
         .load(bronze_path + "sales")
)

display(sales_bronze_df)